[toc]

# Tensorflow Embedding Lookup

embedding lookup 有两种理解方式，一种是将其理解为找到离散特征的嵌入表示。
另一种理解是矩阵乘法在稀疏数据下的特殊情况。

## 稀疏矩阵相乘的特殊情况

假设我们有一个稀疏矩阵

In [172]:
x = tf.constant([[1, 0, 0], [0, 0, 1], [0, 1, 0]])
w = tf.constant([[3], [4], [5]])
tf.matmul(x, w)

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[3],
       [5],
       [4]], dtype=int32)>

这个可以转化为 embedding_lookup 的情况。结果和矩阵乘法是相同的。

In [185]:
label_x = tf.argmax(x)
tf.nn.embedding_lookup(w, label_x)

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[3],
       [5],
       [4]], dtype=int32)>

## 一维情况

假设我们有一个权重矩阵

In [ ]:
import tensorflow as tf

embeddings = tf.constant([
    [0.1, 0.1, 0.1, 0.1],
    [0.2, 0.2, 0.2, 0.2],
    [0.3, 0.3, 0.3, 0.3],
    [0.4, 0.4, 0.4, 0.4],
])

我们的数据是包含两个样本和一个离散特征

In [157]:
features = tf.constant([2,3])

In [158]:
tf.nn.embedding_lookup(embeddings, features)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.3, 0.3, 0.3, 0.3],
       [0.4, 0.4, 0.4, 0.4]], dtype=float32)>

embedding_lookup 相当于 one-hot 之后进行矩阵乘法，好处是不需要真的进行 one-hot 操作。因为 one-hot 出来的矩阵会占很多空间。

也就是说，**`embedding_lookup` 可以看作一种针对特征情况的一种高效的矩阵乘法。**

上面的 embedding_lookup 的过程相当于做了下面的矩阵乘法。

In [159]:
one_hot = tf.one_hot(features, depth=4)
print(one_hot)
tf.matmul(one_hot, embeddings)

tf.Tensor(
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]], shape=(2, 4), dtype=float32)


<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0.3, 0.3, 0.3, 0.3],
       [0.4, 0.4, 0.4, 0.4]], dtype=float32)>

## 多维的情况

还是上面的 embedding 矩阵，现在添加一个特征。离散特征

In [154]:
features = tf.constant([[2, 1],
                        [3, 1]])

tf.nn.embedding_lookup(embeddings, features)

<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[0.3, 0.3, 0.3, 0.3],
        [0.2, 0.2, 0.2, 0.2]],

       [[0.4, 0.4, 0.4, 0.4],
        [0.2, 0.2, 0.2, 0.2]]], dtype=float32)>

In [155]:
one_hot = tf.one_hot(features, depth=4)
print(one_hot)
tf.matmul(one_hot, embeddings)

tf.Tensor(
[[[0. 0. 1. 0.]
  [0. 1. 0. 0.]]

 [[0. 0. 0. 1.]
  [0. 1. 0. 0.]]], shape=(2, 2, 4), dtype=float32)


<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[0.3, 0.3, 0.3, 0.3],
        [0.2, 0.2, 0.2, 0.2]],

       [[0.4, 0.4, 0.4, 0.4],
        [0.2, 0.2, 0.2, 0.2]]], dtype=float32)>

## 关于 embedding lookup 的研究

### method1 直接乘法

假设我们有数据如下， 其中第一列是一个连续特征，后两列是离散特征。

In [168]:
import tensorflow as tf

x = tf.constant([
                [0.1, 1, 0],
                [0.2, 2, 1],
                [0.3, 0, 2]]
)

我们一般是会对离散特征进行 one_hot，

In [163]:
sparse1 = tf.one_hot(tf.cast(x[:, 1], dtype=tf.int32), 3)
sparse2 = tf.one_hot(tf.cast(x[:, 2], dtype=tf.int32), 3)

In [165]:
one_hot = tf.concat([x[:, :1], sparse1, sparse2], axis=1)
print(one_hot)

tf.Tensor(
[[0.1 0.  1.  0.  1.  0.  0. ]
 [0.2 0.  0.  1.  0.  1.  0. ]
 [0.3 1.  0.  0.  0.  0.  1. ]], shape=(3, 7), dtype=float32)


权重矩阵为

In [166]:
w = tf.range(1, 8, dtype=tf.float32)[:, tf.newaxis]
print(w)

tf.Tensor(
[[1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]], shape=(7, 1), dtype=float32)


In [169]:
tf.matmul(one_hot, w)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[ 8.1],
       [10.2],
       [ 9.3]], dtype=float32)>

### method2 embedding lookup

要使用 embedding_lookup，需要把所有的离散变量统一编码。

In [111]:
import tensorflow as tf

x = tf.constant([
                [0.1, 1, 0],
                [0.2, 2, 1],
                [0.3, 0, 2]]
)

x_sparse = x[:, 1:]
idx = 0

x_sparse_new = np.zeros_like(x_sparse, dtype=np.int32)
val2idx = {}
for j in range(2):
    val2idx[j] = {}
    for i in range(3):
        val = x_sparse[i, j].numpy()
        if val not in val2idx[j]:
            val2idx[j][val] = idx
            idx += 1  
        x_sparse_new[i, j] = val2idx[j][val]
        
x_sparse_new

array([[0, 3],
       [1, 4],
       [2, 5]], dtype=int32)

此时，对应的权重矩阵分别是

In [136]:
w_dense = tf.constant([[0.1], [0.2], [0.3]], dtype=tf.float32)
w_sparse = tf.constant([[3], [4], [2], [5], [6], [7]], dtype=tf.float32)

In [137]:
tf.reduce_sum(tf.nn.embedding_lookup(w_sparse, x_sparse_new), axis=1) + w_dense

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[ 8.1],
       [10.2],
       [ 9.3]], dtype=float32)>